# Composing Contracts

Toying with ideas from [Composing Contracts:An Adventure in Financial Engineering](https://www.cs.tufts.edu/~nr/cs257/archive/simon-peyton-jones/contracts.pdf) by Jones, Eber and Seward.

In [8]:
from dataclasses import dataclass
from abc import ABC
from datetime import datetime

In [11]:
class Observable(ABC):
    pass

In [12]:
@dataclass
class Konst(Observable):
    constant: float

In [5]:
class Contract(ABC):
    pass

In [10]:
@dataclass
class Zero(Contract):
    pass

@dataclass
class One(Contract):
    currency: str

@dataclass
class Give(Contract):
    contract: Contract

@dataclass
class And(Contract):
    contract1: Contract
    contract2: Contract

@dataclass
class Or(Contract):
    contract1: Contract
    contract2: Contract

@dataclass
class Truncate(Contract):
    horizon: datetime
    contract: Contract

@dataclass
class Then(Contract):
    contract1: Contract
    contract2: Contract

@dataclass
class Scale(Contract):
    observable: Observable
    contract: Contract

@dataclass
class Get(Contract):
    contract: Contract

@dataclass
class Anytime(Contract):
    contract: Contract

In [13]:
def scaleK(constant: float, contract: Contract):
    return Scale(Konst(constant), contract)

In [14]:
def zcb(maturity: datetime, notional: float, currency: str):
    return scaleK(notional, Get(Truncate(maturity, One(currency))))

In [ ]:
zcb()